In [1]:
# ! pip install pandas
import pandas as pd

In [ ]:
# df1 = pd.read_csv('comments_data/comments/df_1.csv', encoding='utf-8')
# df2 = pd.read_csv('comments_data/comments/df_2.csv', encoding='utf-8')
# df3 = pd.read_csv('comments_data/comments/df_3.csv', encoding='utf-8')
# dfw = pd.read_csv('comments_data/comments/df_w.csv', encoding='utf-8')

# df = pd.concat([df1, df2, df3, dfw], ignore_index=True)
# print(len(df))

160307


### Clean data

In [15]:
df1 = pd.read_csv('comments/face2face_comments.csv', encoding='utf-8')
df2 = pd.read_csv('comments/v68_comments.csv', encoding='utf-8')

df = pd.concat([df1, df2], ignore_index=True)
print(len(df))

110804


In [ ]:
# def process_merged_comments(df):
#     cleaned_rows = []
#     i = 0
#     total_rows = len(df)

#     while i < total_rows:
#         row = df.iloc[i]
#         title = str(row['video_title']).strip()
#         comment = str(row['comment_text']).strip() if pd.notna(row['comment_text']) else ""

#         is_main = title.startswith("【#賀瓏夜夜秀】") and comment != ""

#         # 如果是主留言且下一行也是主留言 → 無需合併
#         if is_main and i + 1 < total_rows:
#             next_row = df.iloc[i + 1]
#             next_title = str(next_row['video_title']).strip()
#             next_comment = str(next_row['comment_text']).strip() if pd.notna(next_row['comment_text']) else ""
#             if next_title.startswith("【#賀瓏夜夜秀】") and next_comment != "":
#                 cleaned_rows.append(row)
#                 i += 1
#                 continue

#         # 如果不是主留言，跳過
#         if not is_main:
#             i += 1
#             continue

#         # 開始合併
#         combined_comment = comment
#         print(f"==> 第{i}行需要合併")
#         meta = {
#             'published_at': row['published_at'],
#             'author_name': row['author_name'],
#             'like_count': row['like_count'],
#             'comment_type': row['comment_type']
#         }

#         j = i + 1
#         while j < total_rows:
#             continuation_row = df.iloc[j]
#             cont_title = str(continuation_row['video_title']).strip()

#             # 若遇到下一個主留言就停止
#             if cont_title.startswith("【#賀瓏夜夜秀】"):
#                 break

#             print(f"合併第{i}行第{j}行")

#             # 如果 continuation 中出現 published_at → 判定為 metadata 錯位行，進行欄位還原，並結束合併
#             if pd.notna(continuation_row['published_at']):
#                 corrected_meta = {
#                     'published_at': continuation_row['comment_text'],
#                     'author_name': continuation_row['published_at'],
#                     'like_count': continuation_row['author_name'],
#                     'comment_type': continuation_row['like_count']
#                 }
#                 for col, val in corrected_meta.items():
#                     if pd.notna(val):
#                         meta[col] = val

#                 if cont_title:
#                     combined_comment += cont_title
#                 j += 1
#                 break

#             # 正常 continuation：只合併 video_title
#             if cont_title:
#                 combined_comment += cont_title
#             j += 1

#         # 組成新 row，並寫入合併後資料與修正後 metadata
#         new_row = row.copy()
#         new_row['comment_text'] = combined_comment
#         for col in meta:
#             new_row[col] = meta[col]

#         cleaned_rows.append(new_row)
#         i = j  # 跳過合併過的行
    
#     print("=== 完成合併 ===")

#     return pd.DataFrame(cleaned_rows)


In [16]:
def process_merged_comments(df):
    cleaned_rows = []
    i = 0
    total_rows = len(df)

    while i < total_rows:
        row = df.iloc[i]
        title = str(row['video_title']).strip()
        comment = str(row['comment_text']).strip() if pd.notna(row['comment_text']) else ""

        is_main = "新聞面對面" in title and comment != ""

        # 如果是主留言且下一行也是主留言 → 無需合併
        if is_main and i + 1 < total_rows:
            next_row = df.iloc[i + 1]
            next_title = str(next_row['video_title']).strip()
            next_comment = str(next_row['comment_text']).strip() if pd.notna(next_row['comment_text']) else ""
            if "新聞面對面" in next_title and next_comment != "":
                cleaned_rows.append(row)
                i += 1
                continue

        # 如果不是主留言，跳過
        if not is_main:
            i += 1
            continue

        # 開始合併
        combined_comment = comment
        print(f"==> 第{i}行需要合併")
        meta = {
            'published_at': row['published_at'],
            'author_name': row['author_name'],
            'like_count': row['like_count'],
            'comment_type': row['comment_type']
        }

        j = i + 1
        while j < total_rows:
            continuation_row = df.iloc[j]
            cont_title = str(continuation_row['video_title']).strip()

            # 若遇到下一個主留言就停止
            if "新聞面對面" in cont_title:
                break

            print(f"合併第{i}行第{j}行")

            # 如果 continuation 中出現 published_at → 判定為 metadata 錯位行，進行欄位還原，並結束合併
            if pd.notna(continuation_row['published_at']):
                corrected_meta = {
                    'published_at': continuation_row['comment_text'],
                    'author_name': continuation_row['published_at'],
                    'like_count': continuation_row['author_name'],
                    'comment_type': continuation_row['like_count']
                }
                for col, val in corrected_meta.items():
                    if pd.notna(val):
                        meta[col] = val

                if cont_title:
                    combined_comment += cont_title
                j += 1
                break

            # 正常 continuation：只合併 video_title
            if cont_title:
                combined_comment += cont_title
            j += 1

        # 組成新 row，並寫入合併後資料與修正後 metadata
        new_row = row.copy()
        new_row['comment_text'] = combined_comment
        for col in meta:
            new_row[col] = meta[col]

        cleaned_rows.append(new_row)
        i = j  # 跳過合併過的行
    
    print("=== 完成合併 ===")

    return pd.DataFrame(cleaned_rows)


In [17]:
df_cleaned = process_merged_comments(df)

==> 第63行需要合併
合併第63行第64行
==> 第231行需要合併
合併第231行第232行
==> 第310行需要合併
合併第310行第311行
合併第310行第312行
==> 第313行需要合併
合併第313行第314行
==> 第479行需要合併
合併第479行第480行
==> 第482行需要合併
合併第482行第483行
合併第482行第484行
==> 第636行需要合併
合併第636行第637行
合併第636行第638行
合併第636行第639行
==> 第798行需要合併
合併第798行第799行
==> 第1225行需要合併
合併第1225行第1226行
合併第1225行第1227行
合併第1225行第1228行
合併第1225行第1229行
==> 第1300行需要合併
合併第1300行第1301行
合併第1300行第1302行
==> 第1351行需要合併
合併第1351行第1352行
合併第1351行第1353行
合併第1351行第1354行
==> 第2034行需要合併
合併第2034行第2035行
==> 第2366行需要合併
合併第2366行第2367行
合併第2366行第2368行
==> 第2442行需要合併
合併第2442行第2443行
合併第2442行第2444行
==> 第2466行需要合併
合併第2466行第2467行
合併第2466行第2468行
合併第2466行第2469行
合併第2466行第2470行
合併第2466行第2471行
合併第2466行第2472行
合併第2466行第2473行
合併第2466行第2474行
合併第2466行第2475行
==> 第2536行需要合併
合併第2536行第2537行
合併第2536行第2538行
合併第2536行第2539行
合併第2536行第2540行
合併第2536行第2541行
合併第2536行第2542行
合併第2536行第2543行
合併第2536行第2544行
合併第2536行第2545行
合併第2536行第2546行
合併第2536行第2547行
合併第2536行第2548行
合併第2536行第2549行
==> 第2939行需要合併
合併第2939行第2940行
==> 第3002行需要合併
合併第3002行第3003行
==> 第3019行需要合併


In [18]:
len(df_cleaned)

105186

In [19]:
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.to_csv('comments/face2face_comments_merged.csv', index=False, encoding='utf-8')

# Count comments

In [20]:
df_group = df_cleaned.groupby(['video_title']).size().reset_index(name='comment_count')
df_group

,video_title,comment_count
0,<br>原因很簡單 執政國家的機會不是給一組還沒準備好的候選人的(選前登記一天都不知道正副搭...,1
1,《 2024兩強一弱？ 藍封殺！ 綠圍堵！ 白營上下亂成一鍋粥？ 》【2023.12.05『...,1125
2,《 2024割喉戰？聚焦「總統們」的居住正義！「房害」效應？ 》【2023.12.28 新...,1683
3,《 2024最大拐點？藍白協商達成6共識！贏家、輸家呼之欲出？ 》【2023.11.16 新...,1298
4,《 2024最大變數？朱柯侯三巨頭昨「會前會」！終局之戰彩蛋？ 》【新聞面對面】2023.1...,537
...,...,...
84,《跟進馬英九！韓國瑜也挺「全民調」！侯友宜hold得住？》【新聞面對面】2023.11.10,1037
85,《馬英九、韓國瑜挺「全民調決勝負」...逼宮朱侯金上車？》【2023.11.13『新聞面對面』】,1009
86,魔鬼藏細節裡？ 朱拋參照日、德式2案？ 柯提全民調讓分？【新聞面對面】2023.11.03,998
87,🔴Live 《 2024關鍵一役！「#新聞面對面」剖析新政局？ 》#2024總統大選 #開票,665


In [10]:
sum(df_group['comment_count'])

30756

In [11]:
df_group.to_csv('comments/face2face_comments_group.csv', index=False, encoding='utf-8')